# Data preparation

Team: HáLab

Team members: Stumphauser Nóra, Lestyan Bence

Import packages

In [121]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [122]:
images = np.load('images.npy')
label = pd.read_csv('label.csv')
label = label.drop(label.columns[0], axis = 1)

In [123]:
x_train, x_test, y_train, y_test = train_test_split(images, label['label'], test_size=0.30, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.50, random_state=42)

In [124]:
print(len(x_train), len(x_valid), len(x_test))

2802 601 601


In [125]:
print(len(y_train), len(y_valid), len(y_test))

2802 601 601


In [126]:
x_train.shape

(2802, 7, 24)

In [127]:
y_train.shape

(2802,)

In [128]:
x_train = x_train.reshape(x_train.shape + (1,))
x_valid = x_valid.reshape(x_valid.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# Last milestone

In [116]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import AUC
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf 

In [117]:
earlyStopping = EarlyStopping(monitor='val_AUC', patience=7, verbose=0, mode='max')
mcp_save = ModelCheckpoint('models/BEST.mdl_wts.hdf5', save_best_only=True, monitor='val_AUC', mode='max')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_AUC', factor=0.1, patience=4, verbose=1, epsilon=1e-4, mode='max')

/home/lestyan/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [199]:
model5 = Sequential()
model5.add(Conv2D(256, kernel_size = (1, 5), strides = (1, 1), activation ='relu',input_shape=(7, 24, 1))) 
#model5.add(Dropout(0.25))
model5.add(MaxPooling2D(pool_size =(1, 2))) 
model5.add(Conv2D(128, kernel_size = (7, 1), strides = (1, 1), activation ='relu'))
model5.add(Dropout(0.2))
model5.add(MaxPooling2D(pool_size =(1, 2))) 
model5.add(Flatten()) 
model5.add(Dense(128, activation ='relu')) 
#model5.add(Dropout(0.2))
model5.add(Dense(64, activation ='relu')) 
#model5.add(Dropout(0.2))
model5.add(Dense(32, activation ='relu')) 
#model5.add(Dropout(0.2)) #-
model5.add(Dense(1, activation ='sigmoid'))

In [200]:
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 7, 20, 256)        1536      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 10, 256)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1, 10, 128)        229504    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1, 10, 128)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 1, 5, 128)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)              

In [201]:
model5.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['AUC'])

In [202]:
model5.fit(x_train, y_train, batch_size=300, epochs=150, validation_data=(x_valid,y_valid), verbose=2, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Train on 2802 samples, validate on 601 samples
Epoch 1/150
2802/2802 - 5s - loss: 2.7064 - AUC: 0.5062 - val_loss: 1.3205 - val_AUC: 0.5154
Epoch 2/150
2802/2802 - 3s - loss: 1.0594 - AUC: 0.5066 - val_loss: 0.7601 - val_AUC: 0.5960
Epoch 3/150
2802/2802 - 2s - loss: 0.7959 - AUC: 0.5400 - val_loss: 0.7084 - val_AUC: 0.6031
Epoch 4/150
2802/2802 - 2s - loss: 0.7109 - AUC: 0.5625 - val_loss: 0.6832 - val_AUC: 0.5987
Epoch 5/150
2802/2802 - 2s - loss: 0.6797 - AUC: 0.6041 - val_loss: 0.6828 - val_AUC: 0.5927
Epoch 6/150
2802/2802 - 2s - loss: 0.6708 - AUC: 0.6156 - val_loss: 0.6828 - val_AUC: 0.6048
Epoch 7/150
2802/2802 - 2s - loss: 0.6629 - AUC: 0.6395 - val_loss: 0.6794 - val_AUC: 0.6057
Epoch 8/150
2802/2802 - 2s - loss: 0.6529 - AUC: 0.6582 - val_loss: 0.6771 - val_AUC: 0.6189
Epoch 9/150
2802/2802 - 2s - loss: 0.6441 - AUC: 0.6755 - val_loss: 0.6769 - val_AUC: 0.6231
Epoch 10/150
2802/2802 - 2s - loss: 0.6454 - AUC: 0.6709 - val_loss: 0.6716 - val_AUC: 0.6287
Epoch 11/150
2802/2802

In [205]:
print('AUC on test data - last step:',model5.evaluate(x_test, y_test, verbose=0)[1])
model4 = tf.keras.models.load_model('models/dl-2_train82_test73_2.h5')
print('AUC on test data - best model:',model4.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.69768685
AUC on test data - best model: 0.7352253


# Keras tuner with RandomSearch

In [130]:
from keras.utils import np_utils
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner import Objective, HyperParameters
import kerastuner as kt
from sklearn import metrics
from keras import backend as K

In [208]:
def create_model(hyperparam):
    model5 = Sequential()
    model5.add(Conv2D(filters=hyperparam.Int('convolution_11',min_value=128, max_value=512, step=128), kernel_size = (1, 5), strides = (1, 1), activation ='relu',input_shape=(7, 24, 1))) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_11',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(MaxPooling2D(pool_size =(1, 2))) 
    model5.add(Conv2D(filters=hyperparam.Int('convolution_22',min_value=128, max_value=512, step=128), kernel_size = (7, 1), strides = (1, 1), activation ='relu'))
    model5.add(Dropout(rate=hyperparam.Float('dropout_22',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(MaxPooling2D(pool_size =(1, 2))) 
    model5.add(Flatten()) 
    model5.add(Dense(units=hyperparam.Int('dense_11',min_value=128, max_value=512, step=128), activation ='relu')) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_33',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(Dense(units=hyperparam.Int('dense_22',min_value=64, max_value=256, step=64), activation ='relu')) 
    model5.add(Dropout(rate=hyperparam.Float('dropout_44',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(Dense(units=hyperparam.Int('dense_33',min_value=32, max_value=128, step=32), activation ='relu')) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_55',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(Dense(1, activation ='sigmoid'))


    model5.compile(optimizer=keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics=['AUC'])
    return model5

In [209]:
tuner_search=RandomSearch(create_model,objective=Objective('val_AUC', direction='max'),max_trials=50,directory='models',project_name="tutorial014")

In [210]:
tuner_search.search(x_train, y_train, batch_size=120, epochs=75,validation_data=(x_valid,y_valid),verbose = 2)

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 5s - loss: 2.1276 - AUC: 0.5138 - val_loss: 0.6815 - val_AUC: 0.5785
Epoch 2/75
2802/2802 - 3s - loss: 0.6972 - AUC: 0.5583 - val_loss: 0.6850 - val_AUC: 0.6173
Epoch 3/75
2802/2802 - 3s - loss: 0.6757 - AUC: 0.6114 - val_loss: 0.6745 - val_AUC: 0.6157
Epoch 4/75
2802/2802 - 3s - loss: 0.6714 - AUC: 0.6266 - val_loss: 0.6707 - val_AUC: 0.6294
Epoch 5/75
2802/2802 - 3s - loss: 0.6474 - AUC: 0.6691 - val_loss: 0.6757 - val_AUC: 0.6564
Epoch 6/75
2802/2802 - 3s - loss: 0.6222 - AUC: 0.7079 - val_loss: 0.6508 - val_AUC: 0.6751
Epoch 7/75
2802/2802 - 3s - loss: 0.5963 - AUC: 0.7404 - val_loss: 0.6590 - val_AUC: 0.6709
Epoch 8/75
2802/2802 - 3s - loss: 0.5764 - AUC: 0.7622 - val_loss: 0.6923 - val_AUC: 0.6759
Epoch 9/75
2802/2802 - 3s - loss: 0.5783 - AUC: 0.7578 - val_loss: 0.6793 - val_AUC: 0.6481
Epoch 10/75
2802/2802 - 3s - loss: 0.5493 - AUC: 0.7874 - val_loss: 0.7391 - val_AUC: 0.6718
Epoch 11/75
2802/2802 - 2s - los

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 11s - loss: 2.2401 - AUC: 0.5017 - val_loss: 0.7012 - val_AUC: 0.5009
Epoch 2/75
2802/2802 - 8s - loss: 0.7168 - AUC: 0.5173 - val_loss: 0.6949 - val_AUC: 0.5533
Epoch 3/75
2802/2802 - 8s - loss: 0.6999 - AUC: 0.4954 - val_loss: 0.6932 - val_AUC: 0.5492
Epoch 4/75
2802/2802 - 8s - loss: 0.6947 - AUC: 0.4830 - val_loss: 0.6931 - val_AUC: 0.5106
Epoch 5/75
2802/2802 - 7s - loss: 0.6930 - AUC: 0.5045 - val_loss: 0.6934 - val_AUC: 0.5363
Epoch 6/75
2802/2802 - 8s - loss: 0.6923 - AUC: 0.5185 - val_loss: 0.6919 - val_AUC: 0.5625
Epoch 7/75
2802/2802 - 8s - loss: 0.6923 - AUC: 0.5233 - val_loss: 0.6926 - val_AUC: 0.5111
Epoch 8/75
2802/2802 - 8s - loss: 0.6873 - AUC: 0.5725 - val_loss: 0.6875 - val_AUC: 0.5792
Epoch 9/75
2802/2802 - 8s - loss: 0.6880 - AUC: 0.5377 - val_loss: 0.6914 - val_AUC: 0.5923
Epoch 10/75
2802/2802 - 8s - loss: 0.6798 - AUC: 0.5940 - val_loss: 0.7048 - val_AUC: 0.6098
Epoch 11/75
2802/2802 - 7s - lo

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 7s - loss: 1.4958 - AUC: 0.5129 - val_loss: 0.7093 - val_AUC: 0.5691
Epoch 2/75
2802/2802 - 4s - loss: 0.7584 - AUC: 0.5256 - val_loss: 0.6896 - val_AUC: 0.5549
Epoch 3/75
2802/2802 - 4s - loss: 0.7095 - AUC: 0.5393 - val_loss: 0.6813 - val_AUC: 0.5953
Epoch 4/75
2802/2802 - 4s - loss: 0.7100 - AUC: 0.5320 - val_loss: 0.6844 - val_AUC: 0.5852
Epoch 5/75
2802/2802 - 4s - loss: 0.6836 - AUC: 0.5853 - val_loss: 0.6720 - val_AUC: 0.6135
Epoch 6/75
2802/2802 - 4s - loss: 0.6913 - AUC: 0.5924 - val_loss: 0.6878 - val_AUC: 0.6265
Epoch 7/75
2802/2802 - 4s - loss: 0.6910 - AUC: 0.5856 - val_loss: 0.6738 - val_AUC: 0.6094
Epoch 8/75
2802/2802 - 4s - loss: 0.6551 - AUC: 0.6510 - val_loss: 0.6726 - val_AUC: 0.6154
Epoch 9/75
2802/2802 - 4s - loss: 0.6676 - AUC: 0.6346 - val_loss: 0.6689 - val_AUC: 0.6320
Epoch 10/75
2802/2802 - 4s - loss: 0.6276 - AUC: 0.6993 - val_loss: 0.6701 - val_AUC: 0.6446
Epoch 11/75
2802/2802 - 4s - los

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 5s - loss: 1.7848 - AUC: 0.5015 - val_loss: 0.6943 - val_AUC: 0.5205
Epoch 2/75
2802/2802 - 3s - loss: 0.7203 - AUC: 0.5066 - val_loss: 0.6897 - val_AUC: 0.5773
Epoch 3/75
2802/2802 - 3s - loss: 0.7032 - AUC: 0.5269 - val_loss: 0.6855 - val_AUC: 0.5899
Epoch 4/75
2802/2802 - 3s - loss: 0.6910 - AUC: 0.5630 - val_loss: 0.7110 - val_AUC: 0.6280
Epoch 5/75
2802/2802 - 3s - loss: 0.6837 - AUC: 0.5874 - val_loss: 0.7307 - val_AUC: 0.6252
Epoch 6/75
2802/2802 - 3s - loss: 0.6726 - AUC: 0.6132 - val_loss: 0.7071 - val_AUC: 0.6195
Epoch 7/75
2802/2802 - 3s - loss: 0.6467 - AUC: 0.6714 - val_loss: 0.6893 - val_AUC: 0.6373
Epoch 8/75
2802/2802 - 3s - loss: 0.6265 - AUC: 0.7021 - val_loss: 0.6736 - val_AUC: 0.6325
Epoch 9/75
2802/2802 - 3s - loss: 0.6285 - AUC: 0.6975 - val_loss: 0.6833 - val_AUC: 0.6221
Epoch 10/75
2802/2802 - 3s - loss: 0.6022 - AUC: 0.7315 - val_loss: 0.6683 - val_AUC: 0.6493
Epoch 11/75
2802/2802 - 3s - los

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 8s - loss: 3.2885 - AUC: 0.4961 - val_loss: 0.6952 - val_AUC: 0.5290
Epoch 2/75
2802/2802 - 5s - loss: 0.7509 - AUC: 0.5084 - val_loss: 0.6945 - val_AUC: 0.5206
Epoch 3/75
2802/2802 - 5s - loss: 0.7047 - AUC: 0.5231 - val_loss: 0.6842 - val_AUC: 0.6073
Epoch 4/75
2802/2802 - 5s - loss: 0.6967 - AUC: 0.5355 - val_loss: 0.6831 - val_AUC: 0.5940
Epoch 5/75
2802/2802 - 5s - loss: 0.6884 - AUC: 0.5663 - val_loss: 0.6776 - val_AUC: 0.6107
Epoch 6/75
2802/2802 - 5s - loss: 0.6801 - AUC: 0.5874 - val_loss: 0.6831 - val_AUC: 0.6052
Epoch 7/75
2802/2802 - 5s - loss: 0.6715 - AUC: 0.6109 - val_loss: 0.6712 - val_AUC: 0.6183
Epoch 8/75
2802/2802 - 5s - loss: 0.6636 - AUC: 0.6365 - val_loss: 0.6711 - val_AUC: 0.6201
Epoch 9/75
2802/2802 - 5s - loss: 0.6603 - AUC: 0.6451 - val_loss: 0.6704 - val_AUC: 0.6187
Epoch 10/75
2802/2802 - 5s - loss: 0.6458 - AUC: 0.6722 - val_loss: 0.6659 - val_AUC: 0.6378
Epoch 11/75
2802/2802 - 5s - los

...

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 16s - loss: 2.5125 - AUC: 0.5088 - val_loss: 0.6869 - val_AUC: 0.5621
Epoch 2/75
2802/2802 - 13s - loss: 0.7510 - AUC: 0.5237 - val_loss: 0.6848 - val_AUC: 0.5886
Epoch 3/75
2802/2802 - 14s - loss: 0.7079 - AUC: 0.5302 - val_loss: 0.6884 - val_AUC: 0.5893
Epoch 4/75
2802/2802 - 14s - loss: 0.6755 - AUC: 0.6008 - val_loss: 0.6860 - val_AUC: 0.5910
Epoch 5/75
2802/2802 - 14s - loss: 0.6752 - AUC: 0.6027 - val_loss: 0.6823 - val_AUC: 0.5913
Epoch 6/75
2802/2802 - 13s - loss: 0.6552 - AUC: 0.6527 - val_loss: 0.6939 - val_AUC: 0.6024
Epoch 7/75
2802/2802 - 13s - loss: 0.6375 - AUC: 0.6825 - val_loss: 0.6881 - val_AUC: 0.6065
Epoch 8/75
2802/2802 - 13s - loss: 0.6130 - AUC: 0.7229 - val_loss: 0.6865 - val_AUC: 0.6360
Epoch 9/75
2802/2802 - 14s - loss: 0.6031 - AUC: 0.7325 - val_loss: 0.6945 - val_AUC: 0.6486
Epoch 10/75
2802/2802 - 13s - loss: 0.5777 - AUC: 0.7645 - val_loss: 0.6824 - val_AUC: 0.6575
Epoch 11/75
2802/2802 

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 9s - loss: 1.8334 - AUC: 0.4966 - val_loss: 0.7249 - val_AUC: 0.5324
Epoch 2/75
2802/2802 - 7s - loss: 0.7670 - AUC: 0.5127 - val_loss: 0.6917 - val_AUC: 0.5379
Epoch 3/75
2802/2802 - 7s - loss: 0.7280 - AUC: 0.4956 - val_loss: 0.7131 - val_AUC: 0.5055
Epoch 4/75
2802/2802 - 7s - loss: 0.7048 - AUC: 0.4958 - val_loss: 0.6900 - val_AUC: 0.5729
Epoch 5/75
2802/2802 - 7s - loss: 0.6991 - AUC: 0.5172 - val_loss: 0.6932 - val_AUC: 0.5406
Epoch 6/75
2802/2802 - 6s - loss: 0.6912 - AUC: 0.5421 - val_loss: 0.6896 - val_AUC: 0.5596
Epoch 7/75
2802/2802 - 6s - loss: 0.6935 - AUC: 0.5422 - val_loss: 0.6910 - val_AUC: 0.5604
Epoch 8/75
2802/2802 - 7s - loss: 0.6850 - AUC: 0.5731 - val_loss: 0.6798 - val_AUC: 0.6078
Epoch 9/75
2802/2802 - 7s - loss: 0.6881 - AUC: 0.5735 - val_loss: 0.6926 - val_AUC: 0.5179
Epoch 10/75
2802/2802 - 7s - loss: 0.6817 - AUC: 0.5806 - val_loss: 0.6914 - val_AUC: 0.5619
Epoch 11/75
2802/2802 - 7s - los

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 12s - loss: 2.4536 - AUC: 0.4956 - val_loss: 0.6904 - val_AUC: 0.5663
Epoch 2/75
2802/2802 - 9s - loss: 0.7081 - AUC: 0.5399 - val_loss: 0.6952 - val_AUC: 0.5825
Epoch 3/75
2802/2802 - 10s - loss: 0.6715 - AUC: 0.6117 - val_loss: 0.6922 - val_AUC: 0.5909
Epoch 4/75
2802/2802 - 10s - loss: 0.6544 - AUC: 0.6543 - val_loss: 0.7051 - val_AUC: 0.6055
Epoch 5/75
2802/2802 - 10s - loss: 0.6390 - AUC: 0.6837 - val_loss: 0.6741 - val_AUC: 0.6252
Epoch 6/75
2802/2802 - 9s - loss: 0.6233 - AUC: 0.7051 - val_loss: 0.6923 - val_AUC: 0.6357
Epoch 7/75
2802/2802 - 9s - loss: 0.6044 - AUC: 0.7302 - val_loss: 0.7107 - val_AUC: 0.6515
Epoch 8/75
2802/2802 - 9s - loss: 0.5803 - AUC: 0.7626 - val_loss: 0.7071 - val_AUC: 0.6490
Epoch 9/75
2802/2802 - 9s - loss: 0.5602 - AUC: 0.7807 - val_loss: 0.6847 - val_AUC: 0.6573
Epoch 10/75
2802/2802 - 9s - loss: 0.5495 - AUC: 0.7920 - val_loss: 0.8527 - val_AUC: 0.6431
Epoch 11/75
2802/2802 - 9s -

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 6s - loss: 2.2561 - AUC: 0.4885 - val_loss: 0.7008 - val_AUC: 0.5420
Epoch 2/75
2802/2802 - 3s - loss: 0.7472 - AUC: 0.4960 - val_loss: 0.6884 - val_AUC: 0.5676
Epoch 3/75
2802/2802 - 3s - loss: 0.6979 - AUC: 0.5225 - val_loss: 0.6865 - val_AUC: 0.5694
Epoch 4/75
2802/2802 - 3s - loss: 0.6924 - AUC: 0.5413 - val_loss: 0.6880 - val_AUC: 0.5682
Epoch 5/75
2802/2802 - 3s - loss: 0.6892 - AUC: 0.5557 - val_loss: 0.6823 - val_AUC: 0.6058
Epoch 6/75
2802/2802 - 3s - loss: 0.6828 - AUC: 0.5780 - val_loss: 0.6753 - val_AUC: 0.6199
Epoch 7/75
2802/2802 - 3s - loss: 0.6726 - AUC: 0.6066 - val_loss: 0.6783 - val_AUC: 0.6078
Epoch 8/75
2802/2802 - 3s - loss: 0.6637 - AUC: 0.6362 - val_loss: 0.6805 - val_AUC: 0.6059
Epoch 9/75
2802/2802 - 3s - loss: 0.6601 - AUC: 0.6478 - val_loss: 0.7030 - val_AUC: 0.6317
Epoch 10/75
2802/2802 - 3s - loss: 0.6549 - AUC: 0.6543 - val_loss: 0.6823 - val_AUC: 0.6146
Epoch 11/75
2802/2802 - 3s - los

INFO:tensorflow:Oracle triggered exit


In [211]:
tuner_search.results_summary()

In [212]:
model6=tuner_search.get_best_models(num_models=1)[0]
model6.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 20, 128)        768       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 10, 128)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 10, 256)        229632    
_________________________________________________________________
dropout (Dropout)            (None, 1, 10, 256)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 5, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [213]:
print('AUC on test data - last step:',model6.evaluate(x_test, y_test, verbose=0)[1])
model_best = tf.keras.models.load_model('models/dl-2_train82_test73_2.h5')
print('AUC on test data - best model:',model_best.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.702556
AUC on test data - best model: 0.7352253


# Keras tuner with Hyperband

In [151]:
def create_model(hyperparam):
    model5 = Sequential()
    model5.add(Conv2D(filters=hyperparam.Int('convolution_11',min_value=128, max_value=512, step=128), kernel_size = (1, 5), strides = (1, 1), activation ='relu',input_shape=(7, 24, 1))) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_11',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(MaxPooling2D(pool_size =(1, 2))) 
    model5.add(Conv2D(filters=hyperparam.Int('convolution_22',min_value=128, max_value=512, step=128), kernel_size = (7, 1), strides = (1, 1), activation ='relu'))
    model5.add(Dropout(rate=hyperparam.Float('dropout_22',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(MaxPooling2D(pool_size =(1, 2))) 
    model5.add(Flatten()) 
    model5.add(Dense(units=hyperparam.Int('dense_11',min_value=128, max_value=512, step=128), activation ='relu')) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_33',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(Dense(units=hyperparam.Int('dense_22',min_value=64, max_value=256, step=64), activation ='relu')) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_44',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(Dense(units=hyperparam.Int('dense_33',min_value=32, max_value=128, step=32), activation ='relu')) 
    #model5.add(Dropout(rate=hyperparam.Float('dropout_55',min_value=0.0,max_value=0.2,default=0.1,step=0.05)))
    model5.add(Dense(1, activation ='sigmoid'))


    model5.compile(optimizer=keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics=['AUC'])
    return model5

In [152]:
tuner_search=Hyperband(create_model,objective=Objective('val_AUC', direction='max'),factor=5,max_epochs=75,directory='models',project_name="tutorial019")

In [153]:
tuner_search.search_space_summary()

In [154]:
tuner_search.search(x_train, y_train, batch_size=120, epochs=75,validation_data=(x_valid,y_valid),verbose = 2)

Train on 2802 samples, validate on 601 samples
Epoch 1/3
2802/2802 - 11s - loss: 1.9565 - AUC: 0.5172 - val_loss: 0.7121 - val_AUC: 0.5549
Epoch 2/3
2802/2802 - 8s - loss: 0.7336 - AUC: 0.5247 - val_loss: 0.7259 - val_AUC: 0.6015
Epoch 3/3
2802/2802 - 8s - loss: 0.6856 - AUC: 0.5743 - val_loss: 0.7234 - val_AUC: 0.6043


Train on 2802 samples, validate on 601 samples
Epoch 1/3
2802/2802 - 8s - loss: 2.5609 - AUC: 0.4957 - val_loss: 0.7829 - val_AUC: 0.5184
Epoch 2/3
2802/2802 - 5s - loss: 0.7150 - AUC: 0.5205 - val_loss: 0.6827 - val_AUC: 0.6187
Epoch 3/3
2802/2802 - 5s - loss: 0.6810 - AUC: 0.5941 - val_loss: 0.8058 - val_AUC: 0.6419


Train on 2802 samples, validate on 601 samples
Epoch 1/3
2802/2802 - 8s - loss: 1.9507 - AUC: 0.4854 - val_loss: 0.8663 - val_AUC: 0.5166
Epoch 2/3
2802/2802 - 5s - loss: 0.7419 - AUC: 0.5254 - val_loss: 0.7290 - val_AUC: 0.5602
Epoch 3/3
2802/2802 - 5s - loss: 0.7010 - AUC: 0.5586 - val_loss: 0.7303 - val_AUC: 0.6244


Train on 2802 samples, validate on 601 samples
Epoch 1/3
2802/2802 - 8s - loss: 2.5202 - AUC: 0.5069 - val_loss: 0.6815 - val_AUC: 0.5941
Epoch 2/3
2802/2802 - 5s - loss: 0.7092 - AUC: 0.5408 - val_loss: 0.6832 - val_AUC: 0.6134
Epoch 3/3
2802/2802 - 5s - loss: 0.7205 - AUC: 0.5558 - val_loss: 0.6737 - val_AUC: 0.6174


Train on 2802 samples, validate on 601 samples
Epoch 1/3
2802/2802 - 15s - loss: 2.2181 - AUC: 0.5045 - val_loss: 0.9534 - val_AUC: 0.5759
Epoch 2/3
2802/2802 - 13s - loss: 0.7568 - AUC: 0.5135 - val_loss: 0.6858 - val_AUC: 0.5832
Epoch 3/3
2802/2802 - 13s - loss: 0.6875 - AUC: 0.5740 - val_loss: 0.6978 - val_AUC: 0.5847


...

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 17s - loss: 1.8394 - AUC: 0.5046 - val_loss: 0.6836 - val_AUC: 0.5790
Epoch 2/75
2802/2802 - 13s - loss: 0.6925 - AUC: 0.5783 - val_loss: 0.6794 - val_AUC: 0.6007
Epoch 3/75
2802/2802 - 13s - loss: 0.6603 - AUC: 0.6392 - val_loss: 0.6972 - val_AUC: 0.5973
Epoch 4/75
2802/2802 - 13s - loss: 0.6564 - AUC: 0.6460 - val_loss: 0.7331 - val_AUC: 0.6229
Epoch 5/75
2802/2802 - 14s - loss: 0.6261 - AUC: 0.7048 - val_loss: 0.6782 - val_AUC: 0.6293
Epoch 6/75
2802/2802 - 13s - loss: 0.6143 - AUC: 0.7154 - val_loss: 0.7029 - val_AUC: 0.6279
Epoch 7/75
2802/2802 - 14s - loss: 0.5856 - AUC: 0.7551 - val_loss: 0.6756 - val_AUC: 0.6502
Epoch 8/75
2802/2802 - 13s - loss: 0.5511 - AUC: 0.7913 - val_loss: 0.7613 - val_AUC: 0.6375
Epoch 9/75
2802/2802 - 13s - loss: 0.5274 - AUC: 0.8125 - val_loss: 0.7234 - val_AUC: 0.6444
Epoch 10/75
2802/2802 - 13s - loss: 0.5067 - AUC: 0.8285 - val_loss: 0.7248 - val_AUC: 0.6543
Epoch 11/75
2802/2802 

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 10s - loss: 3.1683 - AUC: 0.4909 - val_loss: 0.8519 - val_AUC: 0.5078
Epoch 2/75
2802/2802 - 7s - loss: 0.7739 - AUC: 0.4962 - val_loss: 0.6922 - val_AUC: 0.5676
Epoch 3/75
2802/2802 - 7s - loss: 0.6910 - AUC: 0.5613 - val_loss: 0.6957 - val_AUC: 0.5829
Epoch 4/75
2802/2802 - 7s - loss: 0.6732 - AUC: 0.6102 - val_loss: 0.6788 - val_AUC: 0.6041
Epoch 5/75
2802/2802 - 7s - loss: 0.6660 - AUC: 0.6313 - val_loss: 0.6826 - val_AUC: 0.6053
Epoch 6/75
2802/2802 - 7s - loss: 0.6595 - AUC: 0.6428 - val_loss: 0.6730 - val_AUC: 0.6132
Epoch 7/75
2802/2802 - 6s - loss: 0.6464 - AUC: 0.6709 - val_loss: 0.6729 - val_AUC: 0.6100
Epoch 8/75
2802/2802 - 7s - loss: 0.6371 - AUC: 0.6904 - val_loss: 0.6728 - val_AUC: 0.6239
Epoch 9/75
2802/2802 - 7s - loss: 0.6158 - AUC: 0.7193 - val_loss: 0.6668 - val_AUC: 0.6446
Epoch 10/75
2802/2802 - 6s - loss: 0.5991 - AUC: 0.7402 - val_loss: 0.6837 - val_AUC: 0.6402
Epoch 11/75
2802/2802 - 6s - lo

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 11s - loss: 2.8628 - AUC: 0.5239 - val_loss: 0.6995 - val_AUC: 0.5730
Epoch 2/75
2802/2802 - 9s - loss: 0.7272 - AUC: 0.5144 - val_loss: 0.6946 - val_AUC: 0.6233
Epoch 3/75
2802/2802 - 9s - loss: 0.6791 - AUC: 0.5934 - val_loss: 0.6754 - val_AUC: 0.6304
Epoch 4/75
2802/2802 - 9s - loss: 0.6700 - AUC: 0.6223 - val_loss: 0.6698 - val_AUC: 0.6433
Epoch 5/75
2802/2802 - 9s - loss: 0.6530 - AUC: 0.6614 - val_loss: 0.6823 - val_AUC: 0.6519
Epoch 6/75
2802/2802 - 9s - loss: 0.6484 - AUC: 0.6643 - val_loss: 0.6593 - val_AUC: 0.6586
Epoch 7/75
2802/2802 - 9s - loss: 0.6218 - AUC: 0.7089 - val_loss: 0.6591 - val_AUC: 0.6663
Epoch 8/75
2802/2802 - 9s - loss: 0.6067 - AUC: 0.7307 - val_loss: 0.6513 - val_AUC: 0.6798
Epoch 9/75
2802/2802 - 8s - loss: 0.5832 - AUC: 0.7543 - val_loss: 0.7011 - val_AUC: 0.6684
Epoch 10/75
2802/2802 - 9s - loss: 0.5622 - AUC: 0.7790 - val_loss: 0.6748 - val_AUC: 0.6797
Epoch 11/75
2802/2802 - 9s - lo

Train on 2802 samples, validate on 601 samples
Epoch 1/75
2802/2802 - 8s - loss: 2.6165 - AUC: 0.5171 - val_loss: 0.6916 - val_AUC: 0.5480
Epoch 2/75
2802/2802 - 6s - loss: 0.7412 - AUC: 0.5079 - val_loss: 0.7242 - val_AUC: 0.5967
Epoch 3/75
2802/2802 - 6s - loss: 0.7281 - AUC: 0.5277 - val_loss: 0.6926 - val_AUC: 0.6021
Epoch 4/75
2802/2802 - 6s - loss: 0.6855 - AUC: 0.5741 - val_loss: 0.6802 - val_AUC: 0.6073
Epoch 5/75
2802/2802 - 6s - loss: 0.6748 - AUC: 0.6032 - val_loss: 0.6821 - val_AUC: 0.6176
Epoch 6/75
2802/2802 - 5s - loss: 0.6598 - AUC: 0.6450 - val_loss: 0.6816 - val_AUC: 0.6141
Epoch 7/75
2802/2802 - 6s - loss: 0.6423 - AUC: 0.6786 - val_loss: 0.7619 - val_AUC: 0.6267
Epoch 8/75
2802/2802 - 6s - loss: 0.6384 - AUC: 0.6821 - val_loss: 0.6985 - val_AUC: 0.6330
Epoch 9/75
2802/2802 - 6s - loss: 0.6210 - AUC: 0.7067 - val_loss: 0.6739 - val_AUC: 0.6454
Epoch 10/75
2802/2802 - 6s - loss: 0.5967 - AUC: 0.7429 - val_loss: 0.7062 - val_AUC: 0.6490
Epoch 11/75
2802/2802 - 6s - los

INFO:tensorflow:Oracle triggered exit


In [155]:
model6=tuner_search.get_best_models(num_models=1)[0]
model6.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 20, 512)        3072      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 10, 512)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 10, 256)        917760    
_________________________________________________________________
dropout (Dropout)            (None, 1, 10, 256)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 5, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 384)               4

In [156]:
print('AUC on test data - last step:',model6.evaluate(x_test, y_test, verbose=0)[1])
model_best = tf.keras.models.load_model('models/dl-2_train82_test73_2.h5')
print('AUC on test data - best model:',model_best.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.7131747
AUC on test data - best model: 0.7352253


# Keras tuner with custom CNN

In [169]:
def create_model(hyperparam):
    #we constructed a "random" model (random = not a very special)
    model = Sequential()
    model.add(Conv2D(filters=hyperparam.Int('convolution_1',min_value=32, max_value=512, step=32), kernel_size =(3, 3), strides =(1, 1), activation ='relu',input_shape=(7, 24, 1))) 
    model.add(Conv2D(filters=hyperparam.Int('convolution_2',min_value=32, max_value=512, step=32), kernel_size =(3, 3), activation ='relu'))
    model.add(Conv2D(filters=hyperparam.Int('convolution_3',min_value=32, max_value=512, step=32), kernel_size =(3, 3), activation ='relu'))
    model.add(Flatten()) 
    model.add(Dense(units=hyperparam.Int('dense_1',min_value=64, max_value=512, step=64), activation ='relu')) 
    model.add(Dense(1, activation ='sigmoid'))
    
    model.compile(optimizer=keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics=['AUC'])
    return model

In [172]:
tuner_search=Hyperband(create_model,objective=Objective('val_AUC', direction='max'),factor=5,max_epochs=75,directory='models',project_name="tutorial021")

In [ ]:
tuner_search.search(x_train, y_train, batch_size=120, epochs=75,validation_data=(x_valid,y_valid),verbose = 2)

Train on 2802 samples, validate on 601 samples
Epoch 1/3
2802/2802 - 20s - loss: 2.6504 - AUC: 0.5206 - val_loss: 0.6931 - val_AUC: 0.5000
Epoch 2/3


In [176]:
model6=tuner_search.get_best_models(num_models=1)[0]
model6.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 5, 22, 128)        1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 20, 448)        516544    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 18, 224)        903392    
_________________________________________________________________
flatten (Flatten)            (None, 4032)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1032448   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,453,921
Trainable params: 2,453,921
Non-trainable params: 0
______________________________________________

In [177]:
print('AUC on test data - last step:',model6.evaluate(x_test, y_test, verbose=0)[1])
model_best = tf.keras.models.load_model('models/dl-2_train82_test73_2.h5')
print('AUC on test data - best model:',model_best.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.70751995
AUC on test data - best model: 0.7352253


# LSTM

In [222]:
from tensorflow.keras.layers import LSTM

In [223]:
x_train, x_test, y_train, y_test = train_test_split(images, label['label'], test_size=0.30, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.50, random_state=42)

In [224]:
model = Sequential()
model.add(LSTM(128, input_shape=(7, 24), activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [225]:
opt = tf.keras.optimizers.Adam(lr=0.01, decay=1e-6)

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['AUC'])

In [226]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 7, 128)            78336     
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [227]:
model.fit(x_train, y_train, batch_size=120, epochs=40, validation_data=(x_valid,y_valid), verbose=2, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Train on 2802 samples, validate on 601 samples
Epoch 1/40
2802/2802 - 9s - loss: 5.4174 - AUC: 0.5041 - val_loss: 0.7033 - val_AUC: 0.5043
Epoch 2/40
2802/2802 - 3s - loss: 0.8612 - AUC: 0.5009 - val_loss: 0.7383 - val_AUC: 0.4564
Epoch 3/40
2802/2802 - 2s - loss: 0.7818 - AUC: 0.5000 - val_loss: 0.6935 - val_AUC: 0.4987
Epoch 4/40
2802/2802 - 2s - loss: 0.6952 - AUC: 0.5050 - val_loss: 0.6936 - val_AUC: 0.5102
Epoch 5/40
2802/2802 - 2s - loss: 0.6947 - AUC: 0.5130 - val_loss: 0.6940 - val_AUC: 0.5000
Epoch 6/40
2802/2802 - 2s - loss: 0.6957 - AUC: 0.4871 - val_loss: 0.6941 - val_AUC: 0.5000
Epoch 7/40
2802/2802 - 2s - loss: 0.7090 - AUC: 0.4706 - val_loss: 0.6937 - val_AUC: 0.5000
Epoch 8/40

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
2802/2802 - 3s - loss: 0.6961 - AUC: 0.4973 - val_loss: 0.6939 - val_AUC: 0.5000
Epoch 9/40
2802/2802 - 2s - loss: 0.6956 - AUC: 0.4995 - val_loss: 0.6939 - val_AUC: 0.5000
Epoch 10/40
2802/2802 - 2s - loss: 0.6929 - 

In [228]:
print('AUC on test data - last step:',model.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.5


# Transfer learning

idea from https://arxiv.org/ftp/arxiv/papers/1901/1901.06091.pdf

In [73]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [74]:
images_to_save = np.load('images.npy')
labels_to_save = pd.read_csv('label.csv')

In [77]:
for i in range(len(images_to_save)):
    if i < 2800:
        if labels_to_save['label'][i] == 0:
            image = Image.fromarray(images_to_save[i,:,:])
            image.save('transferlr/train/nochurn/nochurn'+str(i)+'.tiff')
        elif labels_to_save['label'][i] == 1:
            image = Image.fromarray(images_to_save[i,:,:])
            image.save('transferlr/train/churn/churn'+str(i)+'.tiff')
    elif i>=2800 and i<3400:
        if labels_to_save['label'][i] == 0:
            image = Image.fromarray(images_to_save[i,:,:])
            image.save('transferlr/valid/nochurn/nochurn'+str(i)+'.tiff')
        elif labels_to_save['label'][i] == 1:
            image = Image.fromarray(images_to_save[i,:,:])
            image.save('transferlr/valid/churn/churn'+str(i)+'.tiff')
    else:
        if labels_to_save['label'][i] == 0:
            image = Image.fromarray(images_to_save[i,:,:])
            image.save('transferlr/test/nochurn/nochurn'+str(i)+'.tiff')
        elif labels_to_save['label'][i] == 1:
            image = Image.fromarray(images_to_save[i,:,:])
            image.save('transferlr/test/churn/churn'+str(i)+'.tiff')

In [78]:
# we have to upsace
img_height=150
img_width=150

In [94]:
# making the datagenerators for train,valid and test data
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory('transferlr/train', target_size=(img_height, img_width), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory('transferlr/valid', target_size=(img_height, img_width), batch_size=20, class_mode='binary')
test_generator = test_datagen.flow_from_directory('transferlr/test', target_size=(img_height, img_width), batch_size=20, class_mode='binary')

Found 2800 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 604 images belonging to 2 classes.


In [95]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import *
from keras.models import *
from keras.layers import Input, Dense
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
from keras.utils.np_utils import to_categorical
from keras.models import load_model

In [96]:
import tensorflow.keras


base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
#add some layers, and a Dense layer, with 3 neuron, because we have 3 classes
model = Model(inputs=base_model.input, outputs=predictions)

In [97]:
for layer in base_model.layers:
    layer.trainable = False

# makeing layers untrainable
# then compile the model, with loss=categorical_crossentropy, because we have 2 classes
model.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')

In [98]:
# fitting the train_generators
model.fit_generator(train_generator,steps_per_epoch=200,validation_data=validation_generator,validation_steps=10,epochs=5)

Epoch 1/5
200/200 [==============================] - 482s 2s/step - loss: 0.7262 - accuracy: 0.5132 - val_loss: 33.6639 - val_accuracy: 0.3950
Epoch 2/5
200/200 [==============================] - 429s 2s/step - loss: 0.6876 - accuracy: 0.5447 - val_loss: 11.8931 - val_accuracy: 0.4500
Epoch 3/5
200/200 [==============================] - 426s 2s/step - loss: 0.6816 - accuracy: 0.5543 - val_loss: 7.7946 - val_accuracy: 0.5850
Epoch 4/5
200/200 [==============================] - 419s 2s/step - loss: 0.6869 - accuracy: 0.5458 - val_loss: 4.7664 - val_accuracy: 0.5400
Epoch 5/5
200/200 [==============================] - 418s 2s/step - loss: 0.6776 - accuracy: 0.5757 - val_loss: 12.7887 - val_accuracy: 0.4100


In [99]:
model.evaluate_generator(test_generator)
print('The Accuracy is:', model.evaluate_generator(test_generator)[1])

The Accuracy is: 0.5513244867324829


In [100]:
# we have more trainable layers
for layer in model.layers[:172]:
       layer.trainable = False
for layer in model.layers[172:]:
       layer.trainable = True

In [102]:
# compile again
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'], loss='binary_crossentropy')

In [103]:
# fitting again
model.fit_generator(train_generator,steps_per_epoch=200,validation_data=validation_generator,validation_steps=10,epochs=5)

Epoch 1/5
200/200 [==============================] - 2228s 11s/step - loss: 0.6675 - accuracy: 0.5918 - val_loss: 533.5838 - val_accuracy: 0.5200
Epoch 2/5
200/200 [==============================] - 2073s 10s/step - loss: 0.6590 - accuracy: 0.6152 - val_loss: 1459.1145 - val_accuracy: 0.5750
Epoch 3/5
200/200 [==============================] - 2088s 10s/step - loss: 0.6511 - accuracy: 0.6292 - val_loss: 7019.3232 - val_accuracy: 0.6000
Epoch 4/5
200/200 [==============================] - 2130s 11s/step - loss: 0.6350 - accuracy: 0.6522 - val_loss: 7305.3022 - val_accuracy: 0.5900
Epoch 5/5
200/200 [==============================] - 2146s 11s/step - loss: 0.6210 - accuracy: 0.6683 - val_loss: 13212.3906 - val_accuracy: 0.5850


In [104]:
print('The Accuracy is:', model.evaluate_generator(test_generator)[1])

The Accuracy is: 0.4486754834651947


In [105]:
base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
#add some layers, and a Dense layer, with 3 neuron, because we have 3 classes
model = Model(inputs=base_model.input, outputs=predictions)

# we have more trainable layers
for layer in model.layers[:172]:
       layer.trainable = False
for layer in model.layers[172:]:
       layer.trainable = True
        
model.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')

In [106]:
model.fit_generator(train_generator,steps_per_epoch=200,validation_data=validation_generator,validation_steps=10,epochs=5)

Epoch 1/5
200/200 [==============================] - 2739s 14s/step - loss: 0.7028 - accuracy: 0.5815 - val_loss: 5082.7095 - val_accuracy: 0.4100
Epoch 2/5
200/200 [==============================] - 2330s 12s/step - loss: 0.6121 - accuracy: 0.6660 - val_loss: 117.8294 - val_accuracy: 0.4150
Epoch 3/5
200/200 [==============================] - 2216s 11s/step - loss: 0.5359 - accuracy: 0.7132 - val_loss: 343.3605 - val_accuracy: 0.4350
Epoch 4/5
200/200 [==============================] - 2214s 11s/step - loss: 0.4789 - accuracy: 0.7592 - val_loss: 373.8839 - val_accuracy: 0.5450
Epoch 5/5
200/200 [==============================] - 2230s 11s/step - loss: 0.4087 - accuracy: 0.8092 - val_loss: 408.2622 - val_accuracy: 0.3900


In [107]:
print('The Accuracy is:', model.evaluate_generator(test_generator)[1])

The Accuracy is: 0.5513244867324829
